In [1]:
'''
对使用TVAE模型生成的数据进行SVM分析得分

'''
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

In [2]:
real_data = pd.read_csv('processed_seattle_vv_21.csv')
real_data = real_data.drop(["UNDERINFLY"],axis=1)
real_data = real_data.drop(['UNDERINFLN'],axis=1)
real_data = pd.get_dummies(real_data,columns=['Severity'],dtype=int)
fake_data = pd.read_csv('generated_data_by_tvae_model.csv',index_col=0)



In [3]:
fake_data

,ADDRTYPEAlley,ADDRTYPEBlock,ADDRTYPEIntersection,COLLISIONTYPEAngles,COLLISIONTYPEHead.On,COLLISIONTYPELeft.Turn,COLLISIONTYPEParked.Car,COLLISIONTYPERear.Ended,COLLISIONTYPERight.Turn,COLLISIONTYPESideswipe,...,LIGHTCONDDark...No.Street.Lights,LIGHTCONDDark...Street.Lights.Off,LIGHTCONDDark...Street.Lights.On,LIGHTCONDDawn,LIGHTCONDDaylight,LIGHTCONDDusk,Severity_Fatality Collision,Severity_Injury Collision,Severity_Property Damage Only Collision,Severity_Serious Injury Collision
0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
9996,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
9997,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
9998,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


In [4]:
real_data

,ADDRTYPEAlley,ADDRTYPEBlock,ADDRTYPEIntersection,COLLISIONTYPEAngles,COLLISIONTYPEHead.On,COLLISIONTYPELeft.Turn,COLLISIONTYPEParked.Car,COLLISIONTYPERear.Ended,COLLISIONTYPERight.Turn,COLLISIONTYPESideswipe,...,LIGHTCONDDark...No.Street.Lights,LIGHTCONDDark...Street.Lights.Off,LIGHTCONDDark...Street.Lights.On,LIGHTCONDDawn,LIGHTCONDDaylight,LIGHTCONDDusk,Severity_Fatality Collision,Severity_Injury Collision,Severity_Property Damage Only Collision,Severity_Serious Injury Collision
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3876,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3877,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
3878,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3879,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0


In [5]:
# 制定label  把最后四维做成一维
# fake_data.iloc[9996,-4:]


real_X = real_data.iloc[:,0:48]
A = np.array([[1],[2],[3],[4]])
# print(A.shape)
real_Y = pd.DataFrame(np.dot(real_data.iloc[:,-4:],A))
# real_Y.shape

fake_X = fake_data.iloc[:,0:48]

fake_Y = pd.DataFrame(np.dot(fake_data.iloc[:,-4:],A))
type(fake_X)


pandas.core.frame.DataFrame

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score,f1_score
import sklearn.svm as svm
x_train, x_test, y_train, y_test = train_test_split(real_X, real_Y, test_size=0.2)

model = svm.SVC(kernel="linear", decision_function_shape="ovr")

x_train.shape

# x_train = x_train.append(fake_X)
# y_train = y_train.append(fake_Y)


(3104, 48)

In [7]:
import time
s = time.time()
model.fit(x_train,y_train)
e = time.time()
print('cost time is :',e-s)

cost time is : 0.16702032089233398


C:\Users\10643\anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [8]:
acu_train = model.score(x_train,y_train)
acu_train

0.6923324742268041

In [9]:
acu_test = model.score(x_test, y_test)
acu_test

0.685971685971686

In [10]:
y_pred = model.predict(x_test)
f1 = f1_score(y_test,y_pred,average="macro")
f1

0.3496957403651116

In [11]:
def svc(kernel):
    return svm.SVC(kernel=kernel, decision_function_shape="ovr")


def nusvc():
    return svm.NuSVC(decision_function_shape="ovr")


def linearsvc():
    return svm.LinearSVC(multi_class="ovr")


def modelist():
    modelist = []
    kernalist = {"linear", "poly", "rbf", "sigmoid"}
    for each in kernalist:
        modelist.append(svc(each))
#     modelist.append(nusvc())
    modelist.append(linearsvc())
    return modelist


def svc_model(model):
    model.fit(x_train, y_train)
    acu_train = model.score(x_train, y_train)
    acu_test = model.score(x_test, y_test)
    y_pred = model.predict(x_test)
    f1 = f1_score(y_test, y_pred, average="macro")
    return acu_train, acu_test, f1




In [12]:
def run_svc_model(modelist):
    result = {"kernel": [],
              "acu_train": [],
              "acu_test": [],
              "f1": []
              }

    for model in modelist:
        acu_train, acu_test, f1 = svc_model(model)
        try:
            result["kernel"].append(model.kernel)
        except:
            result["kernel"].append(None)
        result["acu_train"].append(acu_train)
        result["acu_test"].append(acu_test)
        result["f1"].append(f1)

    return pd.DataFrame(result)

run_svc_model(modelist())


C:\Users\10643\anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\10643\anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\10643\anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\10643\anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

,kernel,acu_train,acu_test,f1
0,sigmoid,0.626289,0.651223,0.385345
1,poly,0.707474,0.679537,0.285623
2,rbf,0.694910,0.682111,0.275956
3,linear,0.692332,0.685972,0.349696
4,None,0.691044,0.684685,0.349600
